### Merge tables and write to gold layer
*this layer removes "favorite_count" as this column generated errors and is not used in the analysis

In [ ]:
from pyspark.sql import DataFrame
from databricks_filesystem import DatabricksFilesystem

adb_fs = DatabricksFilesystem(dbutils=dbutils)

file_list = adb_fs.filesystem_list(filesystem_path="mnt/stackoverflow/silver") #This one lists them all

filepaths = []

for filepath in file_list:
    if '.parquet' in filepath:
        filepaths.append("/".join(filepath.split("/")[:-1]))

# print(f'FILTERED LIST: {filepaths}')
# print('')

tablenames = ['badges', 'comments', 'posthistory', 'posts', 'tags', 'users', 'votes'] #TODO udpate only if stackoverflow archive
                                                                                    # adds new table type, which is unlikely

for tablename in tablenames:
    merged_df = None
    print(f"Writing to ./{tablename}")
    for filepath in filepaths:
        if filepath.split('/')[-1] == tablename:
            df = spark.read.format("delta").load(filepath)
            for col in df.columns:
                if 'favorite_count' in col:
                    df = df.drop(col)
            # If merged_df is empty, assign the first dataframe to it
            if merged_df is None:
                merged_df = df
            else:
                # Union (merge) the DataFrames
                merged_df = merged_df.unionByName(df)
    dest = f'/mnt/stackoverflow/gold/stackoverflow/{tablename}/' #/dbfs
    merged_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(dest)
    print(f"Completed writing to ./{tablename}")
    print('')


Writing to ./badges
Completed writing to ./badges

Writing to ./comments
Completed writing to ./comments

Writing to ./posthistory
Completed writing to ./posthistory

Writing to ./posts
Completed writing to ./posts

Writing to ./tags
Completed writing to ./tags

Writing to ./users
Completed writing to ./users

Writing to ./votes
Completed writing to ./votes

